# 06 - Handling Images

In [ ]:
%run ../beamline_configuration.py

Loading metadata history from /home/tcaswell/.config/bluesky/bluesky_history.db


## Array data in Theory

We can read more than just scalar data using Bluesky.  For handling
non-scalar data (such as from imaging detectors or MCAs) we do not
directly store the data in the databroker, instead we store _pointer_
to where the data is and how to access it.  This design allows us to buffer the user from such mundane details as what the filename and format of the underlying data is.  
This is also motivated in part by performance concerns both at the database level, you do not want to put gigabytes of binary data into a database, and at the collection level, we do not want to put Bluesky between detectors and disk. 

In short, these _pointers_ allow *DataBroker* to do the file I/O work of opening and extracting data from disk and returns to you numpy arrays.  For more [details about how this works](https://nsls-ii.github.io/databroker/assets.html) see the DataBroker documentation.


## Array data in practice

Lets take a single frame of a low signal-to-noise detector to see how this works

In [ ]:
RE(bps.mv(spot.exp, .005))  # low exposure time so we will get significant noise
RE(bps.mv(mtr_spotx, 0, mtr_spoty, 0))  # set position to dead center

uid, = RE(bp.count([spot, I], num=1))
h_one = db[uid]

If we look at this with `table`:

In [ ]:
h_one.table()

We see a uid string in the column where we expect our image to be!  Taking a look at the `'spot_img'`'s entry in the descriptor

In [ ]:
h_one.descriptors[0]['data_keys']['spot_img']

we see that the expeted shape is `(480, 640)` and `'external'` key indicates that these strings are keys to trade to the *DataBroker* for the actual data. To ask
*DataBroker* to fetch the image data from disk we can pass the optional kwarg `fill`
to `table`

In [ ]:
h_one.table(fill=True)

In [ ]:
h_one.table(fill=True)['spot_img'][1]

but it is embedded in a Pandas data frame.  The `Header` has the
``data`` method which pulls out one column of the stream (and defaults to
`fill=True`).

The object returned by `h.data` is a *generator* which will lazily return
one value at a time.  To grab just the first image we can use `next`

In [ ]:
im = next(h_one.data('spot_img'))
fig, ax = plt.subplots()
im_ = ax.imshow(im)
fig.colorbar(im_)

You have already been working with *generator* functions as under-the-hood the built in  *BlueSky* plans are all generators.  For more details on generators see [the BlueSky appendix](https://nsls-ii.github.io/bluesky/appendix.html) or google ["James Powell generators youtube"](https://www.google.com/search?q=james+powell+generators+youtube).

If we know there is exactly 1 image we can unpack it like:

In [ ]:
im, = h_one.data('spot_img')

To play with generators a bit more, lets 

In [ ]:
uid, = RE(bp.count([spot, I], num=5))
h_few = db[uid]

If there is more than one we can use `list`, `np.vstack`, or
generalize unpacking to pull all of the images

In [ ]:
im_list = list(h_few.data('spot_img'))
im_stack = np.stack(h_few.data('spot_img'))
im1, *rest = h_few.data('spot_img')

We can also iterate through all of them with a `for` loop


In [ ]:
for j, im in enumerate(h_few.data('spot_img')):
    print(f'frame {j} has max {im.max()}')

This has the nice feature that there is only ever 1 frame in memory at a time.  For these 5 small images, this is not a huge issue, but this technique can allow you to process data significantly bigger than your available memory.

## Intermission

Take a few minutes to play the the 4 ways to get all of the images out of the generator

## Summing Images

Now lets take a bunch of images to improve the statistics!


In [ ]:
uid, = RE(bp.count([spot, I], num=150))
h_lots = db[uid]

We can again use =next= to peek at the first image

In [ ]:
im = next(h_lots.data('spot_img'))
fig, ax = plt.subplots()
im_ = ax.imshow(im)
fig.colorbar(im_)



We can then grab the whole stack, average them to one image and
display the result:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
im_stack = np.stack(h_lots.data('spot_img'))

vmin = im_stack.min()
vmax = im_stack.max()

im1 = ax1.imshow(im_stack[0], vmin=vmin, vmax=vmax)
im2 = ax2.imshow(im_stack.mean(axis=0), vmin=vmin, vmax=vmax)

ax1.set_title('1 frame')
ax2.set_title('mean of stack')



`data` can pull out any column.  If you want to access both the beam
current and the image you could do:

In [ ]:
out = np.zeros(h_lots.descriptors[0]['data_keys']['spot_img']['shape'])
j = 0
for cur, im in zip(h_lots.data('I'), h_lots.data('spot_img')):
    out += im / cur
    j += 1

out /= j


However, if you need to access more than one key, it may be better to
use the *Event* documents directly.

In [ ]:
RE(bps.mv(spot.exp, 5))

uid, = RE(bp.spiral_fermat([I, spot], mtr_spotx, mtr_spoty, 0, 0, 75, 75, 3, 1))
h_spiral = db[uid]


In [ ]:

def tri_helper(df, ax, title):
    ax.tricontour(df['x'], df['y'], df['c'], 10, linewidths=0.5, colors='k')
    t = ax.tricontourf(df['x'], df['y'], df['c'], 10)
    ax.set_title(title)
    ax.set_aspect('equal')

    

In [ ]:
import pandas as pd

ret = []
for ev in h_spiral.events(fill=True):
    data = ev['data']
    im = data['spot_img']

    ret.append({'x': data['motor_spotx'],
                'y': data['motor_spoty'],
                'c': np.mean(im[im > 1000])})

df = pd.DataFrame(ret)



In [ ]:
import pandas as pd

ret = []
for ev in h_spiral.events(fill=True):
    data = ev['data']
    im = data['spot_img']

    ret.append({'x': data['motor_spotx'],
                'y': data['motor_spoty'],
                'c': np.mean(im[im > 1000]) / data['I']})

df_normed = pd.DataFrame(ret)



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
tri_helper(df, ax1, 'un normalized')
tri_helper(df_normed, ax2, 'normalized')